In [4]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [5]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('mmdetection/configs/yolox/yolox_s.py')

root='../dataset/'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
#cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.pipeline[1]['img_scale'] = (640,640) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4

cfg.seed=2021
cfg.gpu_ids = [1]
#cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'
#cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [6]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [7]:
epoch = 'best_bbox_mAP_50_epoch_50'

In [8]:
import torch

In [9]:
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
model.eval()

YOLOX(
  (backbone): CSPDarknet(
    (stem): Focus(
      (conv): ConvModule(
        (conv): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (activate): Swish()
      )
    )
    (stage1): Sequential(
      (0): ConvModule(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (activate): Swish()
      )
      (1): CSPLayer(
        (main_conv): ConvModule(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (activate): Swish()
        )
        (short_conv): ConvModule(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, 

In [10]:
checkpoint = torch.load(checkpoint_path)['state_dict']

In [258]:
#for name, value in model.modules():
# new_checkpoint = {}
# for name,_ in model.named_parameters():
#     new_checkpoint[name] = checkpoint["ema_"+name.replace('.','_')]
#     #print(name)

In [18]:
model.load_state_dict(checkpoint,strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['ema_backbone_stem_conv_conv_weight', 'ema_backbone_stem_conv_bn_weight', 'ema_backbone_stem_conv_bn_bias', 'ema_backbone_stem_conv_bn_running_mean', 'ema_backbone_stem_conv_bn_running_var', 'ema_backbone_stem_conv_bn_num_batches_tracked', 'ema_backbone_stage1_0_conv_weight', 'ema_backbone_stage1_0_bn_weight', 'ema_backbone_stage1_0_bn_bias', 'ema_backbone_stage1_0_bn_running_mean', 'ema_backbone_stage1_0_bn_running_var', 'ema_backbone_stage1_0_bn_num_batches_tracked', 'ema_backbone_stage1_1_main_conv_conv_weight', 'ema_backbone_stage1_1_main_conv_bn_weight', 'ema_backbone_stage1_1_main_conv_bn_bias', 'ema_backbone_stage1_1_main_conv_bn_running_mean', 'ema_backbone_stage1_1_main_conv_bn_running_var', 'ema_backbone_stage1_1_main_conv_bn_num_batches_tracked', 'ema_backbone_stage1_1_short_conv_conv_weight', 'ema_backbone_stage1_1_short_conv_bn_weight', 'ema_backbone_stage1_1_short_conv_bn_bias', 'ema_backbone_stage1_1_short_conv_bn_runni

In [19]:
# checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

In [20]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 35.1 task/s, elapsed: 139s, ETA:     0s

In [21]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.16s)
creating index...
index created!


,PredictionString,image_id
0,0 0.1711541 246.26852 683.2528 300.9548 745.11...,test/0000.jpg
1,0 0.060039125 134.66731 4.548865 499.29453 272...,test/0001.jpg
2,0 0.04390883 -4.683862 228.17928 150.37723 374...,test/0002.jpg
3,0 0.0799115 -0.952771 431.28375 121.85231 865....,test/0003.jpg
4,0 0.20143685 185.20773 235.82104 858.46014 808...,test/0004.jpg
